Neo4J v053 Test2 - Sample Based Analysis V01 (Node-Based Results Only)
================

## Set up the environment


### Install py2neo for querying Neo4J 

In [21]:
#!pip3 install py2neo

### Import Packages

In [22]:
from py2neo import Graph
import pandas as pd
import numpy as np
import subprocess
import matplotlib 
import matplotlib.pyplot as plt
import seaborn as sns

#plt.style.use('fivethirtyeight')
#plt.style.use('ggplot')
pd.set_option('display.float_format', lambda x: '%.f' % x)

### Load Neo4J DB

In [23]:
graph = Graph("bolt://35.230.80.57:7687", auth=("neo4j", "bZjcc1XTd71ft2oVGj7A8aU8UkttdBhRAecV4x6USC3wpeOLmCmfCAH2bAMllQQlwtzCbzLuX1TgoX-Enc7MUA"))

## Generate Sample Status Table

### The number of nodes (Fastq, Ubam, Vcf, Cram, Crai) 

In [24]:
# Fastq
query = "Match (j:Json:FromPersonalis), (f:Fastq) WHERE f.sample = j.sample RETURN j.sample AS sample, count(f) AS fastq"
num_fastq = graph.run(query).to_data_frame()
num_fastq.set_index('sample')

# Ubam
query = "Match (j:Json:FromPersonalis), (u:Ubam) WHERE u.sample = j.sample RETURN DISTINCT j.sample AS sample, count(u) AS ubam"
num_ubam = graph.run(query).to_data_frame()
num_ubam.set_index('sample')

# Vcf
query = "Match (j:Json:FromPersonalis), (v:Merged:Vcf) WHERE v.sample = j.sample RETURN DISTINCT j.sample AS sample, count(v) AS vcf"
num_vcf = graph.run(query).to_data_frame()
num_vcf.set_index('sample')

# Cram
query = "Match (j:Json:FromPersonalis), (cm:Cram) WHERE cm.sample = j.sample RETURN DISTINCT j.sample AS sample, count(cm) AS cram"
num_cram = graph.run(query).to_data_frame()
num_cram.set_index('sample')

# Crai
query = "Match (j:Json:FromPersonalis), (ci:Crai) WHERE ci.sample = j.sample RETURN DISTINCT j.sample AS sample, count(ci) AS crai"
num_crai = graph.run(query).to_data_frame()
num_crai.set_index('sample')
print("")

### The number of jobs (FQ2U, GATK) 

In [25]:
# Fq2u
query = "Match (j:Json:FromPersonalis), (e:Job:Dsub {name:'fastq-to-ubam'}) WHERE e.sample = j.sample RETURN j.sample AS sample, count(e) AS fq2u"
num_fq2u = graph.run(query).to_data_frame()
num_fq2u.set_index('sample')

# Gatk
query = "Match (j:Json:FromPersonalis), (g:Job:Cromwell) WHERE g.sample = j.sample RETURN j.sample AS sample, count(g) AS gatk"
num_gatk = graph.run(query).to_data_frame()
num_gatk.set_index('sample')
print("")

### Merge all node and job dfs to one df

In [26]:
#-- only nodes
#sample_qc_df=num_fastq.merge(num_ubam, how='outer').merge(num_vcf, how='outer').merge(num_cram, how='outer').merge(num_crai, how='outer')
#columnlist=["sample","fastq","ubam","vcf","cram","crai"]

#-- nodes and jobs
sample_qc_df=num_fastq.merge(num_ubam, how='outer').merge(num_vcf, how='outer').merge(num_cram, how='outer').merge(num_crai, how='outer').merge(num_fq2u,how='outer').merge(num_gatk,how='outer')
columnlist=["sample","fastq","ubam","vcf","cram","crai","fq2u","gatk"]
numsample=len(sample_qc_df)
print("The number of samples : " + str(numsample) + "\n")

sample_qc_df=sample_qc_df[columnlist]
sample_qc_df.fillna(0,inplace=True)
#sample_qc_df.head(3)

The number of samples : 1330



### Classification based on sample status and success.

In [27]:
##-- Passed

#- 1. Successful jobs
sample_qc_df.loc[(sample_qc_df['ubam']==sample_qc_df['fastq']/2)&(sample_qc_df['vcf']==1)&(sample_qc_df['cram']==1) \
                                   &(sample_qc_df['crai']==1)&(sample_qc_df['fq2u']==sample_qc_df['fastq']/2)&(sample_qc_df['gatk']==1),'status']="success"
#- 2. Duplicated jobs
sample_qc_df.loc[(sample_qc_df['ubam']==sample_qc_df['fastq']/2)&(sample_qc_df['vcf']==1)&(sample_qc_df['cram']==1) \
                                   &(sample_qc_df['crai']==1)&((sample_qc_df['fq2u']>sample_qc_df['fastq']/2)|(sample_qc_df['gatk']>1)),'status']="duplicated jobs"
#- 3. Duplicated jobs and nodes
sample_qc_df.loc[((sample_qc_df['ubam']>sample_qc_df['fastq']/2)|(sample_qc_df['vcf']>1)|(sample_qc_df['cram']>1) \
                                 |(sample_qc_df['crai']>1))&((sample_qc_df['fq2u']>=sample_qc_df['fastq']/2)|(sample_qc_df['gatk']>=1)),'status']="duplicated nodes"
#- Success 
sample_qc_df.loc[(sample_qc_df['status'].isin(["success", "duplicated jobs", "duplicated nodes"])), 'success'] = "pass"


##-- failed

#- 4. failed fq2u jobs
sample_qc_df.loc[((sample_qc_df['ubam']<sample_qc_df['fastq']/2)&(sample_qc_df['vcf']==0)&(sample_qc_df['cram']==0) \
                                   &(sample_qc_df['crai']==0))&((sample_qc_df['fq2u']>=sample_qc_df['fastq']/2)|(sample_qc_df['gatk']==0)),'status']="failed fq2u"
#- 5. failed gatk jobs
sample_qc_df.loc[(sample_qc_df['ubam']==sample_qc_df['fastq']/2)&((sample_qc_df['vcf']<1)|(sample_qc_df['cram']<1) \
                                   |(sample_qc_df['crai']<1))&((sample_qc_df['fq2u']>=sample_qc_df['fastq']/2)&(sample_qc_df['gatk']>=1)),'status']="failed gatk"
#- 6. no gatk jobs
sample_qc_df.loc[((sample_qc_df['ubam']==sample_qc_df['fastq']/2)&(sample_qc_df['vcf']==0)&(sample_qc_df['cram']==0) \
                                   &(sample_qc_df['crai']==0))&((sample_qc_df['fq2u']>=sample_qc_df['fastq']/2)&(sample_qc_df['gatk']==0)),'status']="no gatk"
#- Fail
sample_qc_df.loc[(sample_qc_df['status'].isin(["failed fq2u", "failed gatk", "no gatk"])), 'success'] = "fail"

##-- Check unclassified samples.
num_unclassified = len(sample_qc_df[sample_qc_df.status.isna()==True])
print("The number of unclassified samples : " + str(num_unclassified)+"\n")

if num_unclassified != 0 :
    display(sample_qc_df[sample_qc_df.status.isna()==True])
    
##-- Display of this table
display(sample_qc_df.head(3))


The number of unclassified samples : 0



,sample,fastq,ubam,vcf,cram,crai,fq2u,gatk,status,success
0,SHIP4946368,8,4,1,1,1,4,1,success,pass
1,SHIP4946372,8,4,1,1,1,4,1,success,pass
2,SHIP4948530,8,4,1,1,1,4,1,success,pass


## Generate Status Table

### The number of samples by status and success

In [28]:
stat_status_qc=sample_qc_df['status'].value_counts().to_frame()
stat_status_qc['rate']=100*(stat_status_qc['status']/numsample)
stat_status_qc=stat_status_qc.reindex(index = ['success', 'duplicated jobs', 'duplicated nodes', 'failed fq2u', 'no gatk', 'failed gatk'])
display(stat_status_qc)

pd.set_option('display.float_format', lambda x: '%.2f' % x)
print("Success Rate : " + str(sum(stat_status_qc['rate'][0:3])) + "%")
print("Failed Rate : " + str(sum(stat_status_qc['rate'][3:6])) + "%")

,status,rate
success,1285,97
duplicated jobs,15,1
duplicated nodes,8,1
failed fq2u,15,1
no gatk,3,0
failed gatk,4,0


Success Rate : 98.34586466165413%
Failed Rate : 1.6541353383458646%


## Check the relationship informaiton and dstat message of successful samples

### fq2u issued relationship of successful samples

In [30]:
success_sample_df = sample_qc_df[sample_qc_df['status']=='success']
print("The number of success samples: " + str(len(success_sample_df)))
success_sample=success_sample_df['sample'].to_list()

query = "Match (:Fastq)-[:INPUT_TO]->(j:Job:Dsub) WHERE not (j)-[:OUTPUT]->(:Ubam) and j.sample IN {} RETURN distinct j.sample AS sample_no_fq2utoubam".format(str(success_sample))
norelation_ubam = graph.run(query).to_data_frame()
print("The number of samples without the relationship between fq2u and ubam: " + str(len(norelation_ubam)))
print("\n")

display(norelation_ubam.head(3))

The number of success samples: 1285
The number of samples without the relationship between fq2u and ubam: 3




,sample_no_fq2utoubam
0,SHIP5119430
1,SHIP5153310
2,SHIP5185260


### gatk issued relationship of successful samples

In [31]:
# success_sample_df = sample_qc_df[sample_qc_df['status']=='success']
# print("The number of success samples: " + str(len(success_sample_df)))
# success_sample=norelation_sample_df['sample'].to_list()

query = "Match (j:Job:Cromewell) WHERE (not (:Ubam)-[:INPUT_TO]->(j) or not (j)-[:OUTPUT]->(:Merged:Vcf) or not (j)-[:OUTPUT]->(:Cram) or not (j)-[:OUTPUT]->(:Crai)) and (j.sample IN {}) RETURN distinct j.sample AS sample_missed_output".format(str(success_sample))
norelation_output = graph.run(query).to_data_frame()
print("The number of samples without the relationship between gatk and outputs: " + str(len(norelation_output)))

display(norelation_output.head(3))
#display(set(norelation_sample)-set(norelation_dstat['sample']))

The number of samples without the relationship between gatk and outputs: 0


""


### Issued dstat message in successful samples

In [32]:
# success_sample_df = sample_qc_df[sample_qc_df['status']=='success']
# print("The number of success samples: " + str(len(success_sample_df)))
# success_sample=norelation_sample_df['sample'].to_list()

query = "Match (j:Job:Dsub)-[:STATUS]->(s:Dstat) WHERE j.sample IN {} and not s.status='SUCCESS' RETURN j.sample AS sample, j.logging As log, s.status AS status, s.statusMessage AS Message".format(str(success_sample))
failed_dstat = graph.run(query).to_data_frame()
print("The number of samples with 'FAILURE' as dstat message  : " + str(len(failed_dstat)))

## Issued Sample List
display(failed_dstat.head(3))

## Classified by Messange
failed_dstat.groupby(['Message','status']).count()

The number of samples with 'FAILURE' as dstat message  : 3


,Message,log,sample,status
0,ServiceException: 401 Anonymous caller does no...,gs://gbsc-gcp-project-mvp-test-from-personalis...,SHIP5169288,FAILURE
1,ServiceException: 401 Anonymous caller does no...,gs://gbsc-gcp-project-mvp-test-from-personalis...,SHIP5153375,FAILURE
2,ServiceException: 401 Anonymous caller does no...,gs://gbsc-gcp-project-mvp-test-from-personalis...,SHIP5169413,FAILURE


,,log,sample
Message,status,,
ServiceException: 401 Anonymous caller does not have storage.objects.list access to gbsc-gcp-project-mvp-test-from-personalis-wgs35-logs.\nServiceException: 401 Anonymous caller does not have storage.objects.list access to gbsc-gcp-project-mvp-test-from-personalis-wgs35-logs.\nServiceException: 401 Anonymous caller does not have storage.objects.list access to gbsc-gcp-project-mvp-test-from-personalis-wgs35-logs.\n,FAILURE,3,3


## Check dstat message of samples with duplicated jobs and nodes

### Duplicated Jobs (Fq2u)

In [33]:
dupjobs_sample_df = sample_qc_df[sample_qc_df['status']=='duplicated jobs']
print("The number of success samples with duplicated jobs: " + str(len(dupjobs_sample_df)))
dupjobs_sample=dupjobs_sample_df['sample'].to_list()

#display(dupjobs_sample_df)

query = "Match (j:Job:Dsub)-[:STATUS]->(s:Dstat) WHERE j.sample IN {} and not s.status='SUCCESS' RETURN j.sample AS sample, j.logging As log, s.status AS status, s.statusMessage AS Message".format(str(dupjobs_sample))
dupfq2u_dstat = graph.run(query).to_data_frame()
print("The number of duplicated fq2u jobs : " + str(len(dupfq2u_dstat)))

## Issued Sample and Job List
display(dupfq2u_dstat.head())

## Classified by Messange
dupfq2u_dstat.groupby(['Message','status']).count()

The number of success samples with duplicated jobs: 15
The number of duplicated fq2u jobs : 47


,Message,log,sample,status
0,Stopped running 'prepare',gs://gbsc-gcp-project-mvp-test-from-personalis...,SHIP4954745,RUNNING
1,Worker 'google-pipelines-worker-6b640db5e96c87...,gs://gbsc-gcp-project-mvp-test-from-personalis...,SHIP4954745,RUNNING
2,worker was terminated,gs://gbsc-gcp-project-mvp-test-from-personalis...,SHIP4954744,FAILURE
3,worker was terminated,gs://gbsc-gcp-project-mvp-test-from-personalis...,SHIP4954745,FAILURE
4,worker was terminated,gs://gbsc-gcp-project-mvp-test-from-personalis...,SHIP5142424,FAILURE


,,log,sample
Message,status,,
Started pulling 'gcr.io/gbsc-gcp-project-mvp-test/broadinstitute/gatk:4.1.0.0',RUNNING,13,13
Started running 'localization',RUNNING,15,15
Started running 'prepare',RUNNING,2,2
Started running 'user-command',RUNNING,1,1
Stopped pulling 'gcr.io/gbsc-gcp-project-mvp-test/broadinstitute/gatk:4.1.0.0',RUNNING,4,4
Stopped running 'prepare',RUNNING,1,1
Worker 'google-pipelines-worker-115ef3db30e81d0fe0ae9ac02676b21b' assigned in 'us-west1-a',RUNNING,1,1
Worker 'google-pipelines-worker-6b640db5e96c874bc90e6972f27ef0ae' assigned in 'us-west1-c',RUNNING,1,1
Worker 'google-pipelines-worker-882a96720b399174d1205054b74f7557' assigned in 'us-west1-a',RUNNING,1,1


### Duplicated Jobs (GATK)

In [34]:
#dupjobs_sample_df = sample_qc_df[sample_qc_df['status']=='duplicated jobs']
print("The number of success samples with duplicated jobs: " + str(len(dupjobs_sample_df)))
#dupjobs_sample=dupjobs_sample_df['sample'].to_list()

#display(dupjobs_sample_df)

query = "Match (j:Job:Cromwell)-[:STATUS]->(s:Dstat) WHERE j.sample IN {} and not s.status='SUCCESS' RETURN j.sample AS sample, j.logging As log, s.status AS status, s.statusMessage AS Message".format(str(dupjobs_sample))
dupgatk_dstat = graph.run(query).to_data_frame()
print("The number of duplicated fq2u jobs : " + str(len(dupgatk_dstat)))
display(dupgatk_dstat.head(3))

dupgatk_dstat.groupby(['Message','status']).count()

The number of success samples with duplicated jobs: 15
The number of duplicated fq2u jobs : 1


,Message,log,sample,status
0,Started running 'user-command',gs://gbsc-gcp-project-mvp-test-from-personalis...,SHIP5141855,RUNNING


,,log,sample
Message,status,,
Started running 'user-command',RUNNING,1,1


### Duplicated Job and Node

In [35]:
dupnodes_sample_df = sample_qc_df[sample_qc_df['status']=='duplicated nodes']
print("The number of success samples with duplicated nodes: " + str(len(dupnodes_sample_df)))
dupnodes_sample=dupnodes_sample_df['sample'].to_list()

display(dupnodes_sample_df)

query = "Match (j:Job:Cromwell)-[:STATUS]->(s:Dstat) WHERE j.sample IN {} and not s.status='SUCCESS' RETURN j.sample AS sample, j.logging As log, s.status AS status, s.statusMessage AS Message".format(str(dupnodes_sample))
dupnodes_dstat = graph.run(query).to_data_frame()
print("The number of duplicated nodes: " + str(len(dupnodes_dstat)))
display(dupnodes_dstat.head(3))

#dupjob_dstat.groupby(['Message','status']).count()

The number of success samples with duplicated nodes: 8


,sample,fastq,ubam,vcf,cram,crai,fq2u,gatk,status,success
97,SHIP5119429,8,5,1.00,1.00,1.00,7,1.00,duplicated nodes,pass
365,SHIP5142661,8,4,1.00,1.00,2.00,4,1.00,duplicated nodes,pass
368,SHIP5142468,8,4,1.00,2.00,1.00,4,1.00,duplicated nodes,pass
633,SHIP5153353,8,4,1.00,1.00,2.00,4,1.00,duplicated nodes,pass
1050,SHIP5169373,8,4,1.00,1.00,2.00,4,1.00,duplicated nodes,pass
1248,SHIP5185273,8,4,1.00,2.00,2.00,4,1.00,duplicated nodes,pass
1313,SHIP5185330,8,4,2.00,1.00,1.00,4,1.00,duplicated nodes,pass
1319,SHIP5185343,8,4,1.00,2.00,1.00,4,1.00,duplicated nodes,pass


The number of duplicated nodes: 0


""


## Check the log list and the dstat message of issued jobs

### Failed Fq2u

In [36]:
failed_fq2u_sample_df = sample_qc_df[sample_qc_df['status']=='failed fq2u']
print("The number of samples with failed fq2u jobs: " + str(len(failed_fq2u_sample_df)))
failed_fq2u_sample=failed_fq2u_sample_df['sample'].to_list()

query = "Match (j:Job:Dsub)-[:STATUS]->(s:Dstat) WHERE j.sample IN {} and not s.status='SUCCESS' RETURN j.sample AS sample, j.logging As log, s.status AS status, s.statusMessage AS Message".format(str(failed_fq2u_sample))
failed_fq2u_dstat = graph.run(query).to_data_frame()
print("The number of duplicated fq2u jobs : " + str(len(failed_fq2u_dstat)))
display(failed_fq2u_dstat.head(3))

failed_fq2u_dstat.groupby('Message').count()

The number of samples with failed fq2u jobs: 15
The number of duplicated fq2u jobs : 15


,Message,log,sample,status
0,to gbsc-gcp-project-mvp-test-from-personalis-...,gs://gbsc-gcp-project-mvp-test-from-personalis...,SHIP5119443,FAILURE
1,ServiceException: 401 Anonymous caller does no...,gs://gbsc-gcp-project-mvp-test-from-personalis...,SHIP5141860,FAILURE
2,ServiceException: 401 Anonymous caller does no...,gs://gbsc-gcp-project-mvp-test-from-personalis...,SHIP5142426,FAILURE


,log,sample,status
Message,,,
to gbsc-gcp-project-mvp-test-from-personalis-wgs35-logs.\nServiceException: 401 Anonymous caller does not have storage.objects.list access to gbsc-gcp-project-mvp-test-from-personalis-wgs35-logs.\nServiceException: 401 Anonymous caller does not have storage.objects.list access to gbsc-gcp-project-mvp-test-from-personalis-wgs35-logs.\nServiceException: 401 Anonymous caller does not have storage.objects.list access to gbsc-gcp-project-mvp-test-from-personalis-wgs35-logs.\nServiceException: 401 Anonymous caller does not have storage.objects.list access to gbsc-gcp-project-mvp-test-from-personalis-wgs35-logs.\nServiceException: 401 Anonymous caller does not have storage.objects.list access to gbsc-gcp-project-mvp-test-from-personalis-wgs35-logs.\nServiceException: 401 Anonymous caller does not have storage.objects.list access to gbsc-gcp-project-mvp-test-from-personalis-wgs35-logs.\nServiceException: 401 Anonymous caller does not have storage.objects.list access to gbsc-gcp-project-mvp-test-from-personalis-wgs35-logs.\n,1,1,1
ServiceException: 401 Anonymous caller does not have storage.objects.list access to gbsc-gcp-project-mvp-test-from-personalis-wgs35-logs.\nServiceException: 401 Anonymous caller does not have storage.objects.list access to gbsc-gcp-project-mvp-test-from-personalis-wgs35-logs.\nServiceException: 401 Anonymous caller does not have storage.objects.list access to gbsc-gcp-project-mvp-test-from-personalis-wgs35-logs.\n,14,14,14


### Failed GATK

In [37]:
failed_gatk_sample_df = sample_qc_df[sample_qc_df['status']=='failed gatk']
print("The number of samples with failed gatk jobs: " + str(len(failed_gatk_sample_df)))
failed_gatk_sample=failed_gatk_sample_df['sample'].to_list()

query = "Match (j:Job:Cromwell)-[:STATUS]->(s:Dstat) WHERE j.sample IN {} and not s.status='SUCCESS' RETURN j.sample AS sample, j.logging As log, s.status AS status, s.statusMessage AS Message".format(str(failed_gatk_sample))
failed_gatk_dstat = graph.run(query).to_data_frame()
print("The number of duplicated gatk jobs : " + str(len(failed_gatk_dstat)))
display(failed_gatk_dstat.head(3))

failed_gatk_dstat.groupby('Message').count()

The number of samples with failed gatk jobs: 4
The number of duplicated gatk jobs : 4


,Message,log,sample,status
0,The assigned worker has failed to complete the...,gs://gbsc-gcp-project-mvp-test-from-personalis...,SHIP5141905,FAILURE
1,Workflow 467872d3-5a11-4f6d-a25d-f9f90e61b72d ...,gs://gbsc-gcp-project-mvp-test-from-personalis...,SHIP5165752,FAILURE
2,Workflow e75a10d0-1e1c-4523-840b-982cea8e506c ...,gs://gbsc-gcp-project-mvp-test-from-personalis...,SHIP5153220,FAILURE


,log,sample,status
Message,,,
The assigned worker has failed to complete the operation,1,1,1
Workflow 467872d3-5a11-4f6d-a25d-f9f90e61b72d transitioned to state Failed\n,1,1,1
Workflow 95c7e923-6249-4b25-a269-05984f4069af transitioned to state Failed\n,1,1,1
Workflow e75a10d0-1e1c-4523-840b-982cea8e506c transitioned to state Failed\n,1,1,1
